In [0]:
%python
spark.sql(F"USE CATALOG {catalog_name}")

In [0]:
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS analysis;
CREATE SCHEMA IF NOT EXISTS gold;

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer (
    payer_id BIGINT NOT NULL COMMENT 'Unique internal identifier for the Payer',
    payer_name STRING COMMENT 'Payers name',
    tax_id_value BIGINT COMMENT 'Payer Tax ID Value',
    parent_payer_id BIGINT COMMENT 'FK to Payer - Recursive for hierarchies',
    last_updated_date DATE NOT NULL COMMENT 'When the Payer record was last updated',
    created_date DATE NOT NULL COMMENT 'Date the Payer record was created',
    status STRING NOT NULL COMMENT 'e.g., Active, Inactive, Suspended, etc.',
    address_location_id BIGINT COMMENT 'FK to AddressLocation table',
    contact_id BIGINT COMMENT 'FK to Contact table',
    identifier_id BIGINT COMMENT 'FK to Identifier table',
    CONSTRAINT pk_payer PRIMARY KEY (payer_id)
)
COMMENT 'Main payer entity containing core payer information and relationships';
"""
)

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer_alt_name (
    alt_name_id BIGINT NOT NULL COMMENT 'Unique internal identifier for the Alternate Name',
    payer_id BIGINT NOT NULL COMMENT 'FK to Payer',
    alt_name STRING NOT NULL COMMENT 'Alternate Name of the Payer',
    effective_date DATE NOT NULL COMMENT 'Alternate Name Start Date',
    end_date DATE COMMENT 'Alternate Name End Date',
    CONSTRAINT pk_payer_alt_name PRIMARY KEY (alt_name_id),
    CONSTRAINT fk_payer_alt_name_payer FOREIGN KEY (payer_id) REFERENCES {catalog_name}.bronze.payer(payer_id)
)
COMMENT 'Alternate names and aliases for payers with effective dates';
"""
)

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer_alt_identifiers (
    identifier_id BIGINT NOT NULL COMMENT 'Unique internal identifier for the identifier',
    payer_id BIGINT NOT NULL COMMENT 'FK to Payer',
    identifier_type_code STRING NOT NULL COMMENT 'FHIR identifier.type.code',
    identifier_value STRING NOT NULL COMMENT 'Actual value of the identifier',
    identifier_system STRING NOT NULL COMMENT 'System Identifer Value - used for FHIR integrations',
    assigner STRING COMMENT 'Issuing Organization',
    effective_date DATE NOT NULL COMMENT 'Date when the identifier became valid',
    end_date DATE COMMENT 'Date when the identifier expires (if applicable)',
    CONSTRAINT pk_payer_alt_identifiers PRIMARY KEY (identifier_id),
    CONSTRAINT fk_payer_alt_identifiers_payer FOREIGN KEY (payer_id) REFERENCES {catalog_name}.bronze.payer(payer_id)
)
COMMENT 'FHIR-compliant external identifiers for payers (NPI, DEA, etc.)';
"""
)

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer_address (
    address_location_id BIGINT NOT NULL COMMENT 'Unique internal identifier for the address and location',
    payer_id BIGINT NOT NULL COMMENT 'FK to Payer',
    use_code STRING NOT NULL COMMENT 'Examples - Work, Home',
    address_line_1 STRING NOT NULL COMMENT 'Street address or P.O. Box',
    address_line_2 STRING COMMENT 'Suite, Floor, Building (if applicable)',
    city STRING NOT NULL COMMENT 'City name',
    state STRING NOT NULL COMMENT 'State or province',
    postal_code STRING NOT NULL COMMENT 'Postal or ZIP code',
    country STRING NOT NULL COMMENT 'Country name',
    effective_date DATE NOT NULL COMMENT 'When the address became valid',
    end_date DATE COMMENT 'When the address is no longer valid',
    CONSTRAINT pk_payer_address PRIMARY KEY (address_location_id),
    CONSTRAINT fk_payer_address_payer FOREIGN KEY (payer_id) REFERENCES {catalog_name}.bronze.payer(payer_id)
)
COMMENT 'Physical and mailing addresses for payers with validity periods';
"""
)

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer_contact_information (
    contact_id BIGINT NOT NULL COMMENT 'Unique internal identifier for the contact',
    payer_id BIGINT NOT NULL COMMENT 'FK to Payer',
    contact_name STRING NOT NULL COMMENT 'Payer Contact Name',
    contact_type STRING NOT NULL COMMENT 'e.g., Personal, Office, Administrative, Emergency',
    phone_number STRING NOT NULL COMMENT 'Primary contact number (mobile, landline, etc.)',
    phone_type STRING COMMENT 'e.g., Mobile, Landline, VoIP',
    phone_extension STRING COMMENT 'Extension number (if applicable)',
    email_address STRING NOT NULL COMMENT 'Email address associated with the Payer',
    email_type STRING COMMENT 'e.g., Personal, Work, Academic',
    fax_number STRING COMMENT 'Fax number for communication',
    preferred_contact_method STRING COMMENT 'e.g., Phone, Email, Text',
    availability_hours STRING COMMENT 'Time range when the Payer is reachable',
    emergency_contact_name STRING COMMENT 'Full name of emergency contact (if applicable)',
    emergency_contact_relationship STRING COMMENT 'e.g., Spouse, Parent, Colleague',
    emergency_contact_phone STRING COMMENT 'Phone number of emergency contact',
    CONSTRAINT pk_payer_contact_information PRIMARY KEY (contact_id),
    CONSTRAINT fk_payer_contact_information_payer FOREIGN KEY (payer_id) REFERENCES {catalog_name}.bronze.payer(payer_id)
)
COMMENT 'Comprehensive contact details including phone, email, and emergency contacts';
"""
)

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer_accreditation (
    accreditation_id BIGINT NOT NULL COMMENT 'Unique internal identifier for Accredidation ID',
    payer_id BIGINT NOT NULL COMMENT 'FK to Payer',
    code STRING NOT NULL COMMENT 'Qualification Code',
    issuer STRING NOT NULL COMMENT 'Qulification Issuer',
    effective_date DATE NOT NULL COMMENT 'When the accredidation became valid',
    end_date DATE COMMENT 'When the accredidation is no longer valid',
    CONSTRAINT pk_payer_accreditation PRIMARY KEY (accreditation_id),
    CONSTRAINT fk_payer_accreditation_payer FOREIGN KEY (payer_id) REFERENCES {catalog_name}.bronze.payer(payer_id)
)
COMMENT 'Payer accreditations, qualifications, and certifications';
"""
)

In [0]:
%python
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS bronze.payer_network (
    payer_network_id BIGINT NOT NULL COMMENT 'Unique internal identifier for the Payer Network ID',
    payer_id BIGINT NOT NULL COMMENT 'FK to Payer',
    network_id BIGINT COMMENT 'Network identifier',
    CONSTRAINT pk_payer_network PRIMARY KEY (payer_network_id),
    CONSTRAINT fk_payer_network_payer FOREIGN KEY (payer_id) REFERENCES {catalog_name}.bronze.payer(payer_id)
)
COMMENT 'Association between payers and their networks';
"""
)